In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import pandas as pd
import os

VGG19 MODEL: https://www.kaggle.com/code/kaledibif/fer-project-with-alexnet-vgg19-and-resnet-18

VGG Based Model: https://www.kaggle.com/code/vumerenko/vgg-based-architecture-0-70-test-acc

VGG16 Resized Model: https://github.com/amilkh/cs230-fer/tree/master/models

https://github.com/CNC-IISER-BHOPAL/Tiny-ImageNet-Visual-Recognition-Challenge-IITK/blob/master/custom_vgg16_for_tiny_imagenet.ipynb

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
train_dir = './data/train'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   validation_split=0.2,
                                   rotation_range=5,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1. / 255,
                                   validation_split=0.2)

In [ ]:
train_dataset = train_datagen.flow_from_directory(directory=train_dir,
                                                  target_size=(48, 48),
                                                  class_mode='categorical',
                                                  subset='training',
                                                  batch_size=64)

In [ ]:
valid_dataset = valid_datagen.flow_from_directory(directory=train_dir,
                                                  target_size=(48, 48),
                                                  class_mode='categorical',
                                                  subset='validation',
                                                  batch_size=64)

In [ ]:
image_shape = (48, 48, 3)
num_classes = 7
epochs = 50
lr = 1e-10

In [ ]:
base_model = VGG16(include_top=False, input_shape=image_shape, weights='imagenet')
model = Sequential()
model.add(base_model)

In [ ]:
for layer in model.layers:
    layer.trainable = False

In [ ]:
model.add(Flatten(input_shape=(2, 2, 512)))

In [ ]:
model.add(Dense(512, activation='relu', name='FC1'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

In [ ]:
model.add(Dense(512, activation='relu', name='FC2'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

In [ ]:
model.add(Dense(num_classes, activation='softmax', name='FC3'))

In [ ]:
model.summary()

In [ ]:
lrd = ReduceLROnPlateau(monitor='val_loss', patience=20, verbose=1, factor=0.50, min_lr=lr)

In [ ]:
mcp = ModelCheckpoint('./weights/vgg16_model.h5', monitor='val_accuracy',
                      mode='max',
                      save_best_only=True)

In [ ]:
es = EarlyStopping(verbose=1, patience=20)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_dataset, validation_data=valid_dataset, epochs=epochs,
                    verbose=1, callbacks=[lrd, mcp, es])

In [ ]:
os.makedirs('./history', exist_ok=True)
file_name = './history/vgg16_hist.csv'
with open(file_name, mode='w') as f:
    pd.DataFrame(history.history).to_csv(f)